In [1]:
import pandas as pd
import numpy as np
import pandas as pd
import datetime
from dateutil.relativedelta import relativedelta

import os, pathlib

Access date from https://www.dropbox.com/scl/fo/3ftmsvrig589dmto7noyq/AO1zPEq1JomguQxosV17ilA?rlkey=3gxva1lpcflzuknj6bxloj7er&st=ns2mhnbi&dl=0

In [2]:
MISMATCH = "mismatch_local_viewser"
indice = 'tnx'

In [3]:

from_viewser_path = pathlib.Path().resolve() / MISMATCH / f'{indice}_viewser.csv'
from_viewser_inpanelfalse = pathlib.Path().resolve() / MISMATCH / f'after_ingesting_{indice}_with_inpanelzeroFALSE.csv'
from_local_path = pathlib.Path().resolve() / 'Complete_local_monthly.csv'

from_viewser = pd.read_csv(from_viewser_path)
from_viewser_withnull = pd.read_csv(from_viewser_inpanelfalse)
from_local = pd.read_csv(from_local_path)

In [8]:
print(from_local.head(5))

     gid  tx10p month_year   combined_gid      tx90p       dtr     rx1day  \
0  78878    0.0 2014-06-01  78878_06-2014  16.000000  2.465752  10.796830   
1  78879    0.0 2014-06-01  78879_06-2014  13.333333  3.607715  10.887121   
2  78880    0.0 2014-06-01  78880_06-2014  13.333333  3.607715  10.887121   
3  78881    0.0 2014-06-01  78881_06-2014  13.333333  3.607715  10.887121   
4  78882    0.0 2014-06-01  78882_06-2014  18.888889  4.465008   9.669706   

         txx        tnx  tn10p      tn90p        tnn        txn     rx5day  \
0  19.496094  16.420166    0.0  23.333332  13.088135  15.542627  36.234747   
1  19.646875  15.190088    0.0  16.666666  11.373682  14.845850  34.061275   
2  19.646875  15.190088    0.0  16.666666  11.373682  14.845850  34.061275   
3  19.646875  15.190088    0.0  16.666666  11.373682  14.845850  34.061275   
4  20.145736  14.886133    0.0  19.444444  11.079289  14.889388  32.381588   

  month  year  pgm  
0    06  2014  414  
1    06  2014  414  
2    

In [4]:
def date_to_views_month_id(date: datetime.date) -> int:
    
    views_start_date = datetime.date(1980, 1, 1)
    r = relativedelta(date, views_start_date)
    return (r.years * 12) + r.months + 1

first we will take a look at the 'local' etccdi climate data which was ingested. In order to accurately compare this dataset with the viewser derived dataset we need to first create a pgm field in the 'local' dataset.

Then, lets display the first five rows to give a perspective of the data. Again, this was the dataset ingested to viewser!

Finally, we will print the field types to expose any discrepency between this dataset and the one sourced from Viewser. 

In [10]:
import pandas as pd
import datetime
from dateutil.relativedelta import relativedelta

# Drop unnecessary columns if they exist
from_local = from_local.drop(columns=['Unnamed: 0', 'unnamed: 0'], errors='ignore')

# Split 'month_year' into 'month' and 'year' columns
from_local[['month', 'year']] = from_local['month_year'].str.split('-', expand=True)

# Convert 'month_year' to datetime format
from_local['month_year'] = pd.to_datetime(from_local['month_year'], format='%m-%Y')

# Apply the date_to_views_month_id function to 'month_year'
from_local['pgm'] = from_local['month_year'].apply(lambda x: date_to_views_month_id(x.date()))

print(from_local.head(5))

for column in from_local.columns:
    print(f"Column '{column}' has data type: {from_local[column].dtype}")


     gid  tx10p month_year   combined_gid      tx90p       dtr     rx1day  \
0  78878    0.0 2014-06-01  78878_06-2014  16.000000  2.465752  10.796830   
1  78879    0.0 2014-06-01  78879_06-2014  13.333333  3.607715  10.887121   
2  78880    0.0 2014-06-01  78880_06-2014  13.333333  3.607715  10.887121   
3  78881    0.0 2014-06-01  78881_06-2014  13.333333  3.607715  10.887121   
4  78882    0.0 2014-06-01  78882_06-2014  18.888889  4.465008   9.669706   

         txx        tnx  tn10p      tn90p        tnn        txn     rx5day  \
0  19.496094  16.420166    0.0  23.333332  13.088135  15.542627  36.234747   
1  19.646875  15.190088    0.0  16.666666  11.373682  14.845850  34.061275   
2  19.646875  15.190088    0.0  16.666666  11.373682  14.845850  34.061275   
3  19.646875  15.190088    0.0  16.666666  11.373682  14.845850  34.061275   
4  20.145736  14.886133    0.0  19.444444  11.079289  14.889388  32.381588   

  month  year  pgm  
0    06  2014  414  
1    06  2014  414  
2    

Now, there are missing data values present in the 'from_viewser' table. I suspect that the null values ACTUALLY
have a corresponding data value in the 'from_local' csv. Lets first gather some summary statistics about these missing 
values to comprehend both the spatial (PG) and temporal (m) extents.

In [11]:
print(list(from_viewser_withnull))

['month_id', 'priogrid_gid', 'max_val_of_daily_min_temp']


In [12]:
pd.set_option('display.max_rows', 600)

# Function to count null values by year for each column
def count_null_by_year(time_field, df):
    null_counts = {}
    for column in df.columns:
        if column == time_field:
            continue
        null_counts[column] = df.groupby(time_field)[column].apply(lambda x: x.isnull().sum())
    return pd.DataFrame(null_counts)

# Call the function
null_counts_by_time = count_null_by_year('month_id',from_viewser_withnull)
print(null_counts_by_time)

# Filter the DataFrame to rows where Column B is null
null_values_df = from_viewser_withnull[from_viewser_withnull['max_val_of_daily_min_temp'].isnull()]
unique_combinations = null_values_df[['month_id', 'priogrid_gid']].drop_duplicates()

display(unique_combinations)
# Extract the values of 'month_id' and 'priogrid_id' from the filtered DataFrame
month_id_values = null_values_df['month_id'].unique()
priogrid_id_values = null_values_df['priogrid_gid'].unique()

len_of_missing_month_id = len(month_id_values)
len_of_missing_priogrid_gid = len(priogrid_id_values)
len_of_missing_viewsercombination = len(unique_combinations)

print(f'the length of missing months is: {len_of_missing_month_id}')
print(f'the length of missing pg values is: {len_of_missing_priogrid_gid}')
print(f'the length of the combination of pg|month values is: {len_of_missing_viewsercombination}')
print("Values of 'month_id' corresponding to null values in Column B:", month_id_values)
print("Values of 'priogrid_id' corresponding to null values in Column B:", priogrid_id_values)


          priogrid_gid  max_val_of_daily_min_temp
month_id                                         
121                  0                          3
122                  0                          3
123                  0                          3
124                  0                          3
125                  0                          3
126                  0                          3
127                  0                          3
128                  0                          3
129                  0                          3
130                  0                          3
131                  0                          3
132                  0                          3
133                  0                          3
134                  0                          3
135                  0                          3
136                  0                          3
137                  0                          3
138                  0                          3


,month_id,priogrid_gid
0,121,62356
2477,121,116373
2478,121,116374
13110,122,62356
15587,122,116373
...,...,...
6450109,612,189793
6450110,612,189804
6450111,612,189805
6450112,612,189806


the length of missing months is: 492
the length of missing pg values is: 13110
the length of the combination of pg|month values is: 907250
Values of 'month_id' corresponding to null values in Column B: [121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138
 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156
 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174
 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192
 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210
 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228
 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246
 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264
 265 266 267 268 269 270 271 272 273 274 275 276 277 278 279 280 281 282
 283 284 285 286 287 288 289 290 291 292 293 294 295 296 297 298 299 300
 301 302 303 304 305 306 307 308 309 310 311 312 313 314 315 316 317

So right away it is very apart there is a large swath of data missing! Lets compare this with the 
data in our 'local' version!

In [13]:
pd.set_option('display.max_rows', 600)

null_counts_from_local = count_null_by_year('pgm', from_local)
print(null_counts_from_local)

     gid  tx10p  month_year  combined_gid  tx90p  dtr  rx1day  txx   tnx  \
pgm                                                                        
121    0      0           0             0      0    0       0    0     0   
122    0      0           0             0      0    0       0    0     0   
123    0      0           0             0      0    0       0    0     0   
124    0      0           0             0      0    0       0    0     0   
125    0      0           0             0      0    0       0    0     0   
126    0      0           0             0      0    0       0    0     0   
127    0      0           0             0      0    0       0    0     0   
128    0      0           0             0      0    0       0    0     0   
129    0      0           0             0      0    0       0    0     0   
130    0      0           0             0      0    0       0    0     0   
131    0      0           0             0      0    0       0    0     0   
132    0    

Now this is very strange, right? we see that the 'from_local' table does in fact provide some 'null' values. However,
this was to be expected. The majority of ETCCDI values were processed at an extent beyond the Prio Grid bounds. A few indices,
including tnx was processed as the true priogrid extent so it is to be expected that some null values will appear in the local dataset.

Now, lets look summarize the missing pgm values.

In order to test this, we will focus just on max_val_of_daily_min_temp, a field called 'tnx' in the local table

In [41]:
null_tnx_values = from_local[from_local['tnx'].isnull()]

# Extract the unique combinations of 'pgm' and 'month_id'
unique_combinations_for_tnx = null_tnx_values[['gid', 'pgm']].drop_duplicates()
length_of_unique_combinations_for_tnx = len(unique_combinations_for_tnx)
print(f'the length of missing gid | pgm values is: {length_of_unique_combinations_for_tnx}')
print(f'...and as a reminder, the length of missing gid | pgm from viewser is: {len_of_missing_viewsercombination}')
remind_me_of_these_combination_values_later = f'the length of missing gid | pgm values is: {length_of_unique_combinations_for_tnx} and as a reminder, the length of missing gid | pgm from viewser is: {len_of_missing_viewsercombination}'
print(unique_combinations_for_tnx)

the length of missing gid | pgm values is: 142321
...and as a reminder, the length of missing gid | pgm from viewser is: 907250
            gid  pgm
4219500   78878  515
4219501   78879  515
4219502   78880  515
4219503   78881  515
4219504   78882  515
...         ...  ...
5972522  158839  433
5972561  158878  433
5972562  158879  433
5972563  159445  433
5972564  159446  433

[142321 rows x 2 columns]


Again, we see there are many more values missing in viewser than in the local dataset. Lets first just 
compare the local and viewser based tables by merging left, both, and right.

In [34]:
# Merge the 'from_local' and 'from_viewser' tables on 'pgm' and 'gid' using an outer join
merged_df = pd.merge(from_local, from_viewser, how='outer', left_on=['pgm', 'gid'], right_on=['priogrid_gid', 'month_id'], indicator=True)

column_from_local = 'tnx'
column_from_viewser = list(from_viewser)[2]

#We want to compare the pg | pgm values that only appear in the local version
# Filter rows that only appear in 'from_local'
only_in_local = merged_df[merged_df['_merge'] == 'left_only']
only_in_local = only_in_local[['pgm', 'gid', column_from_local, 'priogrid_gid', 'month_id', column_from_viewser]]
print("Values that only appear in 'from_local':")
print(only_in_local)

#only_in_local.to_csv(f'/Users/gbenz/Desktop/ETCCDI_June_06/mismatch_local_viewser/{column_from_local}_only_in_local.csv')

#We want to compare the pg | pgm values that match both local and viewser
# Filter rows that match in both 'from_local' and 'from_viewser'
match_in_both = merged_df[merged_df['_merge'] == 'both']
match_in_both = match_in_both[['pgm', 'gid', column_from_local, 'priogrid_gid', 'month_id', column_from_viewser]]
print("Values that appear in both:")
print(match_in_both)

#We want to compare the pg | pgm values that only appear in viewser
only_in_viewser = merged_df[merged_df['_merge'] == 'right_only']
only_in_viewser = only_in_viewser[['pgm', 'gid', column_from_local, 'priogrid_gid', 'month_id', column_from_viewser]]
print("Values that only appear in 'from_viewser':")
print(only_in_viewser)
#only_in_viewser.to_csv(f'/Users/gbenz/Desktop/ETCCDI_June_06/mismatch_local_viewser/{column_from_local}_only_in_viewser.csv')
#From this... we want to understand why/if the data in viewser is different from the local version. 

Values that only appear in 'from_local':
           pgm       gid        tnx  priogrid_gid  month_id  \
0        414.0   78878.0  16.420166           NaN       NaN   
1        414.0   78879.0  15.190088           NaN       NaN   
2        414.0   78880.0  15.190088           NaN       NaN   
3        414.0   78881.0  15.190088           NaN       NaN   
4        414.0   78882.0  14.886133           NaN       NaN   
...        ...       ...        ...           ...       ...   
5972586  433.0  159468.0  19.861084           NaN       NaN   
5972587  433.0  159469.0  20.151726           NaN       NaN   
5972588  433.0  159470.0  20.733008           NaN       NaN   
5972589  433.0  159471.0  20.733008           NaN       NaN   
5972590  433.0  159472.0  20.733008           NaN       NaN   

         max_val_of_daily_min_temp  
0                              NaN  
1                              NaN  
2                              NaN  
3                              NaN  
4                

... this is strange, right? This suggests an additional issue. From viewser, there are numerous rows that do in fact 
present non-null values that do not match any data from the 'local' table. This is very odd. We will investigate this further...

Lets look specifically at the null values derived from viewser. So that we are
not working with all etccdi values in the 'from_local' dataset we will slice out 'tnx':

Because there are more null values in viewser than in local we should expect to see at least the total number of null values in the local table match the result from this forthcoming merge. If these numbers are different.... well... hmmm.

In [42]:
pd.set_option('display.max_rows', 600)

#this was called earlier...
unique_combinations = null_values_df[['month_id', 'priogrid_gid']].drop_duplicates()

#we only want to focus on tnx which corresponds to 'max_of_daily_min'
tnx_from_local = from_local[['gid','pgm','tnx']]
print(tnx_from_local.head(5))

# changing column names to match the 'from_local' table
unique_combinations = unique_combinations.rename(columns={'month_id': 'pgm', 'priogrid_gid': 'gid'})

# now -- merge to find matching combinations
match_missing_viewser_with_local = pd.merge(unique_combinations, tnx_from_local, how='inner', on=['pgm', 'gid'])
print(len(match_missing_viewser_with_local))
print(remind_me_of_these_combination_values_later)

print("Matching combinations of 'pgm' and 'gid':")
print(match_missing_viewser_with_local.head(5))


     gid  pgm        tnx
0  78878  414  16.420166
1  78879  414  15.190088
2  78880  414  15.190088
3  78881  414  15.190088
4  78882  414  14.886133
22946
the length of missing gid | pgm values is: 142321 and as a reminder, the length of missing gid | pgm from viewser is: 907250
Matching combinations of 'pgm' and 'gid':
   pgm    gid  tnx
0  421  80317  NaN
1  421  80323  NaN
2  421  80324  NaN
3  421  80325  NaN
4  421  80326  NaN


Now, lets revisit those mismatched values from viewser that contained data (non-null) that could
not find a respective match in local. This is now moving beyond our intial query to locate the bug
causing ingester to read in certain PG values as null -- but perhaps it is related. 

In [52]:
# lets first select 5 random rows
random_rows = only_in_viewser.sample(n=105)

# we only need the 'month_id' and 'priogrid_gid' columns
random_combinations = random_rows[['month_id', 'priogrid_gid']]

#now, lets see if those values actually DO exist in the 'local' dataset:
print(random_combinations)

for index, row in random_combinations.iterrows():
    a = row['priogrid_gid']
    b = row['month_id']
    
    # Subset from_local where pgm == a and gid == b
    subset_df = tnx_from_local[(tnx_from_local['gid'] == a) & (tnx_from_local['pgm'] == b)]
    
    # Print the subset DataFrame
    print(f"Subset for month_id={a} and priogrid_gid={b}:")
    print(subset_df)
    print()

         month_id  priogrid_gid
6899592     602.0      149372.0
6740935     574.0      153061.0
6200217     465.0      116328.0
6053496     429.0      162457.0
6167800     448.0      181085.0
6604664     535.0      150880.0
6060501     432.0      123525.0
6649676     545.0      173867.0
6931687     605.0      179731.0
6142313     445.0       92577.0
6698164     564.0      143020.0
6370734     501.0      170282.0
6635944     543.0      170361.0
6056444     429.0      178223.0
6755193     575.0      158840.0
6690692     560.0      172445.0
6495200     518.0      126389.0
6369757     501.0      165265.0
6402596     504.0      125680.0
6520629     520.0      116323.0
6546577     523.0      112728.0
6338373     492.0      145789.0
6859325     592.0      148022.0
6468986     510.0      129281.0
6767845     578.0      152268.0
6891107     601.0      172462.0
6534795     522.0      129265.0
6406735     504.0      153723.0
6194012     450.0      181070.0
6511439     519.0      148670.0
6221226 

If you adjust the parameters of the sample (n=) you will eventually find that there
are values located in the 'local' table that when merged, were identified to be
only present in viewser.

In [4]:
print(from_viewser.head(10))
print(list(from_local))

   month_id  priogrid_gid  max_val_of_daily_min_temp
0       121         62356                   0.000000
1       121        116373                   0.000000
2       121        116374                   0.000000
3       121        157359                   4.804101
4       121        157360                   4.804101
5       121        157361                   4.804101
6       121        158079                   4.804101
7       121        158080                   4.804101
8       121        158081                   4.804101
9       121        158799                   4.804101
['gid', 'tx10p', 'month_year', 'combined_gid', 'tx90p', 'dtr', 'rx1day', 'txx', 'tnx', 'tn10p', 'tn90p', 'tnn', 'txn', 'rx5day', 'month', 'year', 'pgm']


In [13]:
# Find gid values in from_viewser that do not appear in from_local
gid_not_in_local = from_viewser[~from_viewser['priogrid_gid'].isin(from_local['gid'])]['priogrid_gid']

# Print the result
print("gid values in 'from_viewser' table that do not appear in 'from_local' table:")
print(gid_not_in_local)

# Create a DataFrame with gid_not_in_local as a single column called 'missing'
missing_df = pd.DataFrame({'missing': gid_not_in_local})

# Print the resulting DataFrame
print(missing_df)
#missing_df.to_csv('/Users/gbenz/Desktop/ETCCDI_June_06/mismatch_local_viewser/tnx_in_viewser_not_local.csv')


gid values in 'from_viewser' table that do not appear in 'from_local' table:
0          62356
1         116373
2         116374
1254       62356
1255      116373
           ...  
973035    116373
973036    116374
973037     62356
973038    116373
973039    116374
Name: priogrid_gid, Length: 1476, dtype: int64
        missing
0         62356
1        116373
2        116374
1254      62356
1255     116373
...         ...
973035   116373
973036   116374
973037    62356
973038   116373
973039   116374

[1476 rows x 1 columns]


As we are working backwards to troubleshoot, the ingester code is now supplied by commented out:

In [ ]:
# path = pathlib.Path().resolve() / DIRECTORY / f'{period}_{temporal}_climate_extremes.csv'


# clim_data = pd.read_csv(path, low_memory=False)
# clim_data['date'] = pd.to_datetime(clim_data['month_year'])
# clim_data = pd.DataFrame.pgm.from_datetime(clim_data, datetime_col='date')

# for useless_column in ['month_year','combined_id','date', 'Unnamed: 0', 'unnamed: 0', 'combined_gid']:
#     try:
#         del clim_data[useless_column]
#     except:
#         pass
# clim_data = clim_data.rename(columns={'gid':'pg_id'}).set_index(['pg_id','month_id']).add_suffix('_hgll').reset_index()
# print(clim_data)


#ingester params:

# if temporal=='Monthly':
#     print("Fetching monthly db ids...")
#     clim_data_to_ingest = clim_data_to_ingest.pgm.db_id().pgm.trim_to_db_extent()
    
#     print("Instantiating writer...")
#     pgm_writer = DBWriter(clim_data_to_ingest, 
#                           level='pgm',
#                           in_panel_wipe = True,
#                           out_panel_wipe = False,
#                           in_panel_zero = False,
#                           out_panel_zero = False)
    
#     print("Writing...")
#     extents = clim_data_to_ingest.month_id.min(), clim_data_to_ingest.month_id.max()
#     print (f"extents={extents}")
#     pgm_writer.set_time_extents_min_max(*extents)
#     pgm_writer.transfer(tname='climate_extrmes')
#     print(f"Done! Transferred {clim_data_to_ingest.shape[0]} rows to PGM.")

# if temporal=='Annual':
#     clim_data['year_id']=clim_data.pgm.year
#     del clim_data['month_id']
#     if not clim_data.pgy.is_unique:
#         raise KeyError("This does not look like an annual dataframe. You have duplicates!")

#     print("Fetching yearly db ids...")
#     clim_data = clim_data.pgy.db_id().pgy.trim_to_db_extent()
#     print("Instantiating writer...")
#     pgy_writer = DBWriter(clim_data, 
#                           level='pgy',
#                           in_panel_wipe = True,
#                           out_panel_wipe = False,
#                           in_panel_zero = True,
#                           out_panel_zero = False)
#     extents = clim_data.year_id.min(), clim_data.year_id.max()
#     print (f"extents={extents}")
#     pgy_writer.set_time_extents_min_max(*extents)
    
#     print("Writing...")
#     pgy_writer.transfer(tname='climate_extrmes')
#     print(f"Done! Transferred {clim_data.shape[0]} rows to PGY.")